# 🔐 XOR + LSB Steganography – Message Hiding (Medium Level - English)

In [1]:
from PIL import Image

def hide_xor_message(image_path, message, key, output_name="B_encoded_medium.png"):
    message_bytes = message.encode('utf-8')
    key_bytes = key.encode('utf-8')
    encrypted = bytes([b ^ key_bytes[i % len(key_bytes)] for i, b in enumerate(message_bytes)])
    binary_msg = ''.join(format(b, '08b') for b in encrypted) + '1111111111111111'

    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    encoded = img.copy()
    data_idx = 0
    for y in range(encoded.size[1]):
        for x in range(encoded.size[0]):
            r, g, b = encoded.getpixel((x, y))
            rgb = [r, g, b]
            for i in range(3):
                if data_idx < len(binary_msg):
                    rgb[i] = (rgb[i] & ~1) | int(binary_msg[data_idx])
                    data_idx += 1
            encoded.putpixel((x, y), tuple(rgb))
            if data_idx >= len(binary_msg):
                encoded.save(output_name)
                print(f"[✓] Message successfully hidden in: {output_name}")
                return

In [ ]:
# Example usage
message = "Bybit attack has been initiated."
key = "secretkey"
hide_xor_message("B.png", message, key)

# 🔓 XOR + LSB Steganography – Message Revealing (Medium Level - English)

In [3]:
from PIL import Image

def reveal_xor_message(image_path, key):
    key_bytes = key.encode('utf-8')
    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    binary_data = ""
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            for color in img.getpixel((x, y)):
                binary_data += str(color & 1)
    byte_data = []
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        if byte == '11111111' and binary_data[i+8:i+16] == '11111111':
            break
        byte_data.append(int(byte, 2))
    encrypted = bytes(byte_data)
    try:
        return bytes([b ^ key_bytes[i % len(key_bytes)] for i, b in enumerate(encrypted)]).decode('utf-8')
    except UnicodeDecodeError:
        return "[❌] Incorrect key or corrupted data."

In [ ]:
# Example usage
key = "secretkey"
print(reveal_xor_message("B_encoded_medium.png", key))